In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, auc, precision_recall_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import mlflow
import mlflow.sklearn
from mlflow.models import ModelSignature
from mlflow.models.signature import infer_signature

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
# Initialize MLflow
mlflow.set_tracking_uri('file:mlruns')  # Local directory for tracking
mlflow.set_experiment('Spam Classifier Experiment')

2025/03/04 23:24:36 INFO mlflow.tracking.fluent: Experiment with name 'Spam Classifier Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///C:/Users/chaud/OneDrive/Desktop/Applied ML/Assignment '
 '2/mlruns/918975700612171805'), creation_time=1741110876391, experiment_id='918975700612171805', last_update_time=1741110876391, lifecycle_stage='active', name='Spam Classifier Experiment', tags={}>

In [18]:
# Load data splits
train = pd.read_csv('train.csv').dropna()
validation = pd.read_csv('validation.csv').dropna()
test = pd.read_csv('test.csv').dropna()

# Vectorizing the text data using TF-IDF
def vectorize_data(train_data, validation_data, test_data):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    X_train = vectorizer.fit_transform(train_data['text'])
    X_validation = vectorizer.transform(validation_data['text'])
    X_test = vectorizer.transform(test_data['text'])
    return X_train, X_validation, X_test, vectorizer

X_train, X_validation, X_test, vectorizer = vectorize_data(train, validation, test)

# Function to calculate AUCPR
def calculate_aucpr(model, X, y_true):
    y_scores = model.predict_proba(X)[:, 1] if hasattr(model, 'predict_proba') else model.decision_function(X)
    precision, recall, _ = precision_recall_curve(y_true, y_scores, pos_label=1)
    aucpr = auc(recall, precision)
    return aucpr

# Model training, tracking, and registration with MLflow
def train_and_track_model(model_name, model, param_grid, X_train, y_train, X_validation, y_validation):
    with mlflow.start_run(run_name=model_name) as run:
        # Hyperparameter tuning with GridSearchCV
        grid_search = GridSearchCV(model, param_grid, scoring='recall', cv=5, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        # Evaluation on validation set
        val_precision, val_recall, val_accuracy = evaluate_model(best_model, X_validation, y_validation)
        aucpr = calculate_aucpr(best_model, X_validation, y_validation)

        print(f'{model_name} - AUCPR: {aucpr:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, Accuracy: {val_accuracy:.4f}')

        # Log parameters, metrics, and model in MLflow
        mlflow.log_params(grid_search.best_params_)
        mlflow.log_metrics({
            'precision': val_precision,
            'recall': val_recall,
            'accuracy': val_accuracy,
            'aucpr': aucpr
        })

        # Log and register the model
        signature = infer_signature(X_validation, best_model.predict(X_validation))
        mlflow.sklearn.log_model(best_model, 'model', signature=signature)
        mlflow.set_tag('model_name', model_name)

        model_uri = f'runs:/{run.info.run_id}/model'
        mlflow.register_model(model_uri=model_uri, name='SpamClassifierModel')

        return best_model

# Evaluate model
def evaluate_model(model, X, y, average='binary'):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    recall = recall_score(y, y_pred, pos_label=1, average=average)
    precision = precision_score(y, y_pred, pos_label=1, average=average)
    return precision, recall, accuracy

In [19]:
# Define models and hyperparameter grids
models = {
    'Naive Bayes': (MultinomialNB(), {'alpha': [0.01, 0.1, 1, 10]}),
    'Logistic Regression': (LogisticRegression(), {'C': [0.01, 0.1, 1, 10], 'solver': ['liblinear']}),
    'SVM': (SVC(probability=True), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']})
}

# Train, track, and register all models
for model_name, (model, param_grid) in models.items():
    train_and_track_model(model_name, model, param_grid, X_train, train['category'], X_validation, validation['category'])

Naive Bayes - AUCPR: 0.9875, Precision: 0.9826, Recall: 0.9113, Accuracy: 0.9844


Successfully registered model 'SpamClassifierModel'.
Created version '1' of model 'SpamClassifierModel'.


Logistic Regression - AUCPR: 0.9892, Precision: 0.9910, Recall: 0.8871, Accuracy: 0.9820


Registered model 'SpamClassifierModel' already exists. Creating a new version of this model...
Created version '2' of model 'SpamClassifierModel'.


SVM - AUCPR: 0.9913, Precision: 0.9825, Recall: 0.9032, Accuracy: 0.9832


Registered model 'SpamClassifierModel' already exists. Creating a new version of this model...
Created version '3' of model 'SpamClassifierModel'.


## Loading SVM Model

In [22]:
logged_model = 'runs:/c306231fc2f34d07ada87e4d43432eaa/model'
model = mlflow.pyfunc.load_model(logged_model)

# Get predictions and convert 'spam' to 1 and 'ham' to 0
y_scores = model.predict(X_test)
y_scores = (y_scores == 1).astype(int)

# Convert true labels to 1 for 'spam' and 0 for 'ham'
y_true = (test['category'] == 1).astype(int)

# Calculate precision, recall, and AUCPR
precision, recall, _ = precision_recall_curve(y_true, y_scores)
aucpr = auc(recall, precision)

# Print the model selection metric AUCPR
print(f"Model AUCPR: {aucpr:.4f}")

Model AUCPR: 0.9541


## Loading Logistic Regression Model

In [23]:
logged_model = 'runs:/7c8ce34cb7b541b4b4933fbe7b794d44/model'
model = mlflow.pyfunc.load_model(logged_model)

# Get predictions and convert 'spam' to 1 and 'ham' to 0
y_scores = model.predict(X_test)
y_scores = (y_scores == 1).astype(int)

# Convert true labels to 1 for 'spam' and 0 for 'ham'
y_true = (test['category'] == 1).astype(int)

# Calculate precision, recall, and AUCPR
precision, recall, _ = precision_recall_curve(y_true, y_scores)
aucpr = auc(recall, precision)

# Print the model selection metric AUCPR
print(f"Model AUCPR: {aucpr:.4f}")

Model AUCPR: 0.9425


## Loading Naive Bayes Model

In [24]:
logged_model = 'runs:/4235cd7e4757424f990eb67d79f2c92b/model'
model = mlflow.pyfunc.load_model(logged_model)

# Get predictions and convert 'spam' to 1 and 'ham' to 0
y_scores = model.predict(X_test)
y_scores = (y_scores == 1).astype(int)

# Convert true labels to 1 for 'spam' and 0 for 'ham'
y_true = (test['category'] == 1).astype(int)

# Calculate precision, recall, and AUCPR
precision, recall, _ = precision_recall_curve(y_true, y_scores)
aucpr = auc(recall, precision)

# Print the model selection metric AUCPR
print(f"Model AUCPR: {aucpr:.4f}")

Model AUCPR: 0.9582


## Best Model: Naive Bayes with AUCPR: 0.9582